In [1]:
import numpy as np
import scipy as sp
import math
import torch
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import optuna
import functools
from sklearn.model_selection import KFold
import time
import torch.autograd as autograd


from typing import Callable
from torch import nn, optim
from data import diffeq, create_trainig_test_set, create_trainig_validation_test_set

import torch.nn.functional as F

In [2]:
#Define the harmonic oscillator

def f(t,x,k):
    #write the function that is on the other side of the differential equation
    return -k*x

def differential_equation_2_order(t, X, k):
    #rewrite d^2x/dt^2 as dy/dt where y=dx/dt
    x, y = X
    dx_dt = y
    dy_dt = f(t,x,k)

    return [dx_dt, dy_dt]

In [3]:
if torch.accelerator.is_available():
    device = torch.accelerator.current_accelerator().type
else:
    device = "cpu"
print(device)

cpu


In [12]:
#Define NN

class HarmonicModel(nn.Module):
    def __init__(self, n_in, hidden_sizes, activation):
        super().__init__()
        act = {
            'tanh': nn.Tanh(),
            'sine': torch.sin,
            'gelu': nn.GELU(),
            'relu': nn.ReLU()
        }

        #allow for different sizes of neurons for each layer
        layer_sizes = [n_in] + hidden_sizes + [1] #becomes for example [4, 64, 32, 16, 1]
        
        self.net = nn.ModuleList([
            nn.Linear(layer_sizes[i], layer_sizes[i+1])
            for i in range(len(layer_sizes)-1)
        ]) #nn.Linear(4, 64), nn.Linear(64, 32), ..., nn.Linear(16, 1)

        self.activation = act[activation] #chooses a value from the dictionary, e.g. act['tanh']

    def forward(self, x):
        for layer in self.net[:-1]:
            if callable(self.activation):  # sine special case
                x = self.activation(layer(x))
            else:
                x = self.activation(layer(x))
        return self.net[-1](x)
    
def loss_harmonic(y_pred, y_exp):
    mse_y = torch.mean((y_pred - y_exp)**2)
    
    return mse_y

#evaluate model
def evaluate_NN(model, data):
    model.eval()
    X_val = data["X"].to(device)
    y_val = data["y_tensor"][:, :, 0].to(device)

    with torch.no_grad():
        y_pred = model(X_val).squeeze(-1)
        loss = loss_harmonic(y_pred, y_val)

    return loss.item()

In [14]:
#Define PINN

class HarmonicModel2(nn.Module):
    def __init__(self, n_in, n_layers, n_neurons, activation):
        super().__init__()
        layers = []
        act = {
            'tanh': nn.Tanh(),
            'sine': torch.sin,
            'gelu': nn.GELU(),
            'relu': nn.ReLU()
        }[activation]
        
        layers.append(nn.Linear(n_in, n_neurons))
        for _ in range(n_layers - 1):
            layers.append(nn.Linear(n_neurons, n_neurons))
        layers.append(nn.Linear(n_neurons, 1))
        
        self.layers = nn.ModuleList(layers)
        self.activation = act

    def forward(self, x):
        for layer in self.layers[:-1]:
            if callable(self.activation):  # sine special case
                x = self.activation(layer(x))
            else:
                x = self.activation(layer(x))
        return self.layers[-1](x)

def loss_PINN2(model, y_pred, y_exp, x_00, x_01, t, k, N_f):
    mse_y = torch.mean((y_pred - y_exp)**2)

    t_f = torch.linspace(0, 100, steps=N_f).repeat(len(t), 1).unsqueeze(-1).requires_grad_(True).to(device)

    n_rep = int(N_f / len(y_exp[0]))

    k_f = k.unsqueeze(-1).repeat(1, n_rep, 1)
    x_00_f = x_00.unsqueeze(-1).repeat(1, n_rep, 1)
    x_01_f = x_01.unsqueeze(-1).repeat(1, n_rep, 1)

    X_f = torch.cat([k_f, x_00_f, x_01_f, t_f], dim=-1)
    
    y_f = model(X_f)
    x_t = autograd.grad(y_f, t_f,
                       grad_outputs=torch.ones_like(y_f),
                       create_graph=True, allow_unused=True)[0]
    
    x_tt = autograd.grad(x_t, t_f,
                        grad_outputs=torch.ones_like(x_t),
                        create_graph=True, allow_unused=True)[0]
    
    # Residual
    f_res = x_tt + k_f*y_f
    mse_f = torch.mean(f_res**2)

    return mse_y + mse_f

#evaluate model
def evaluate_PINN(model, data, N_f=1000):
    model.eval()

    X = data["X"].to(device)
    y_exp = data["y_tensor"][0:, 0:, 0].to(device)

    y_pred = model(X).squeeze(-1)
    t = X[0:, 0:, 3]
    k = X[0:, 0:, 0]
    x_00 = X[0:, 0:, 1]
    x_01 = X[0:, 0:, 2]

    with torch.no_grad():
        y_pred = model(X).squeeze(-1)
        loss = loss_PINN2(model, y_pred, y_exp, x_00, x_01, t, k, N_f)

    return loss.item()


In [ ]:
#load models

# Model class must be defined somewhere
model_NN = torch.load("optimized_model_NN.pt", weights_only=False)
model_NN.eval()  # put in evaluation mode
model_PINN = torch.load("optimized_model_PINN.pt", weights_only=False, map_location=torch.device('cpu'))
model_PINN.eval()  # put in evaluation mode

In [16]:
#Define dataset to compare models

torch.manual_seed(0) #get the same random numbers everytime you run the code
harmonic_osc = diffeq(differential_equation_2_order, 2, 1)

t_span = (0, 10) #wanted timespan
n_step = 100 #number of steps to train the data (number of initial conditions)
n_data = 1000 #number of data per step (per initial condition, this number of datapoints to train, validate, test data)

#validation set will be used to choose hyperparameters, decide when to stop training, compare multiple models fairly and detect overfitting
train, test = create_trainig_test_set(harmonic_osc,
                                      t_span=t_span,
                                      n_steps=n_step,
                                      n_data=n_data,
                                      coeff_test=0.3,
                                      method="RK45",
                                      device=device,
                                      seed=0)

In [17]:
#evaluate models

#evaluate
loss_opt_NN = evaluate_NN(model_NN, data=test)
print(f"The test loss on the optimized model is {loss_opt_NN}")

loss_opt_PINN = evaluate_PINN(model_PINN, data=test)
print(f"The test loss on the optimized model is {loss_opt_PINN}")

The test loss on the optimized model is 0.07714784890413284


AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

# Performance on noisy data

In [18]:
#Compare the performance on noisy data

def add_noise_to_dataset(dataset, tensor_keys=['y_tensor'], noise_std=0.05, seed=None):
    """
    Add Gaussian noise to specified tensors in a dataset.

    Args:
        dataset (dict): Dictionary of tensors, e.g., {'y_tensor': ..., 't_tensor': ...}
        tensor_keys (list): Which tensors to add noise to (usually target/output tensors)
        noise_std (float): Standard deviation of Gaussian noise
        seed (int, optional): Random seed for reproducibility
    Returns:
        noisy_dataset (dict): Copy of dataset with added noise
    """
    if seed is not None:
        torch.manual_seed(seed)
    
    noisy_dataset = {k: v.clone() for k, v in dataset.items()}  # copy original tensors
    
    for key in tensor_keys:
        noisy_dataset[key] += torch.randn_like(dataset[key]) * noise_std
    
    return noisy_dataset

In [ ]:
#Noisy data (create only 1 set to check the model performances)
train_no_noise, test_no_noise = create_trainig_test_set(harmonic_osc,
                                                        t_span=(0, 10),
                                                        n_steps=100,
                                                        n_data=1,
                                                        coeff_valtest=[0.699, 0.001, 0.3],
                                                        method="RK45",
                                                        device=device,
                                                        seed=0)

# Example: add noise with std=0.1 to y_tensor
noise_std = 0.5
train_noise = add_noise_to_dataset(train_no_noise, tensor_keys=['y_tensor'], noise_std=noise_std, seed=0)
test_noise = add_noise_to_dataset(test_no_noise, tensor_keys=['y_tensor'], noise_std=noise_std, seed=0)

# test dataset bayes
X = test_noise["X"]

#get prediction for NN and PINN
y_pred_noise_NN = model_NN(X).squeeze(-1)
y_pred_noise_PINN = model_PINN(X).squeeze(-1)

test_loss_noise_NN = evaluate_NN(model_NN, data=test_noise)
print(f"Test Loss Random optimization: {test_loss_noise_NN:.4f}")
test_loss_noise_PINN = evaluate_NN(model_PINN, data=test_noise)
print(f"Test Loss Random optimization: {test_loss_noise_PINN:.4f}")

In [ ]:
#Plot the results
n = 0

y_pred_noise_NN.to("cpu")
y_pred_noise_PINN.to("cpu")
test_noise["t_tensor"].to("cpu")

#Solve numerically
y0 = [test_noise["x0_tensor"][n][0], test_noise["x0_tensor"][n][1]]  # initial condition [x0, v0]
t_span = test_noise["t_tensor"][n][0], test_noise["t_tensor"][n][-1]
w = test_noise["args_tensor"][n]

diff = differential_equation_2_order
sol = sp.integrate.solve_ivp(diff, t_span, y0, t_eval=test_noise["t_tensor"][n], method='RK45', args=(w))

# Create figure with GridSpec
fig = plt.figure(figsize=(7, 9))
gs = fig.add_gridspec(2, 1, height_ratios=[3, 1], hspace=0)

#create ratios for lower plot
slice = 5 #so that the plot is clear
ratio_an_num = (test_no_noise["y_tensor"][n][0:, 0]/sol.y[0])[::slice]
ratio_an_NN = (test_no_noise["y_tensor"][n][0:, 0]/y_pred_noise_NN.cpu().detach()[n])[::slice]
ratio_an_PINN = (test_no_noise["y_tensor"][n][0:, 0]/y_pred_noise_PINN.cpu().detach()[n])[::slice]

# Main plot
ax_main = fig.add_subplot(gs[0])
ax_main.scatter(test_noise["t_tensor"][n], test_noise["y_tensor"][n][0:, 0], label="Analytical solution") #analytical
ax_main.plot(sol.t, sol.y[0], label='Numerical solution', color = 'green') #numerical
ax_main.plot(test_noise["t_tensor"][n], y_pred_noise_NN.cpu().detach()[n], color="blue", label="Predictions NN") #NN ML
ax_main.plot(test_noise["t_tensor"][n], y_pred_noise_PINN.cpu().detach()[n], color="red", label="Predictions PINN") #PINN ML
ax_main.set_ylabel('Predicted values')
ax_main.legend()
ax_main.grid(True)
ax_main.tick_params(bottom=False)  # remove x-axis ticks for main plot

# Ratio / comparison plot
ax_ratio = fig.add_subplot(gs[1], sharex=ax_main)
ax_ratio.plot(test_noise["t_tensor"][n][::slice], ratio_an_num, '.', label='Analytical/Numerical', color='green')
ax_ratio.plot(test_noise["t_tensor"][n][::slice], ratio_an_NN, '.', label='Analytical/NN', color='blue')
ax_ratio.plot(test_noise["t_tensor"][n][::slice], ratio_an_PINN, '.', label='Analytical/PINN', color='red')
ax_ratio.axhline(1, color='gray', linestyle='--')
ax_ratio.set_xlabel('time (s)')
ax_ratio.set_ylabel('Ratio')
ax_ratio.set_ylim(0, 2)
ax_ratio.legend()
plt.show()

# Generalization

In [ ]:
#Generalization
train_generalization, test_generalization = create_trainig_test_set(harmonic_osc,
                                                                    t_span=(0, 50),
                                                                    n_steps=200,
                                                                    n_data=1,
                                                                    coeff_test=0.3,
                                                                    method="RK45",
                                                                    device=device,
                                                                    seed=0)
# test dataset bayes
X = test_generalization["X"]
y_pred_generalization_NN = model_NN(X).squeeze(-1)
y_pred_generalization_PINN = model_PINN(X).squeeze(-1)


test_loss_generalization_NN = evaluate_NN(model_NN, data=test_generalization)
print(f"Test Loss Random optimization: {test_loss_generalization_NN:.4f}")
test_loss_generalization_PINN = evaluate_NN(model_PINN, data=test_generalization)
print(f"Test Loss Random optimization: {test_loss_generalization_PINN:.4f}")

In [ ]:
n = 0

y_pred_generalization_NN.to("cpu")
y_pred_generalization_PINN.to("cpu")
test_generalization["t_tensor"].to("cpu")

#Solve numerically
y0 = [test_generalization["x0_tensor"][n][0], test_generalization["x0_tensor"][n][1]]  # initial condition [x0, v0]
t_span = test_generalization["t_tensor"][n][0], test_generalization["t_tensor"][n][-1]
w = test_generalization["args_tensor"][n]

diff = differential_equation_2_order
sol = sp.integrate.solve_ivp(diff, t_span, y0, t_eval=test_generalization["t_tensor"][n], method='RK45', args=(w))

# Create figure with GridSpec
fig = plt.figure(figsize=(7, 9))
gs = fig.add_gridspec(2, 1, height_ratios=[3, 1], hspace=0)

#create ratios for lower plot
slice = 5
ratio_an_num = (test_generalization["y_tensor"][n][0:, 0]/sol.y[0])[::slice]
ratio_an_NN = (test_generalization["y_tensor"][n][0:, 0]/y_pred_generalization_NN.cpu().detach()[n])[::slice]
ratio_an_PINN = (test_generalization["y_tensor"][n][0:, 0]/y_pred_generalization_PINN.cpu().detach()[n])[::slice]

# Main plot
ax_main = fig.add_subplot(gs[0])
ax_main.scatter(test_generalization["t_tensor"][n], test_generalization["y_tensor"][n][0:, 0], label="Analytical solution") #analytical
ax_main.plot(sol.t, sol.y[0], label='Numerical solution', color = 'green') #numerical
ax_main.plot(test_generalization["t_tensor"][n], y_pred_generalization_NN.cpu().detach()[n], color="blue", label="Predictions NN") #NN ML
ax_main.plot(test_generalization["t_tensor"][n], y_pred_generalization_PINN.cpu().detach()[n], color="red", label="Predictions PINN") #PINN ML
ax_main.set_ylabel('Distance')
ax_main.legend()
ax_main.grid(True)
ax_main.tick_params(bottom=False)  # remove x-axis ticks for main plot

# Ratio / comparison plot
ax_ratio = fig.add_subplot(gs[1], sharex=ax_main)
ax_ratio.plot(test_generalization["t_tensor"][n][::slice], ratio_an_num, '.', label='Analytical/Numerical', color='green')
ax_ratio.plot(test_generalization["t_tensor"][n][::slice], ratio_an_NN, '.', label='Analytical/NN', color='blue')
ax_ratio.plot(test_generalization["t_tensor"][n][::slice], ratio_an_PINN, '.', label='Analytical/PINN', color='red')
ax_ratio.axhline(1, color='gray', linestyle='--')
ax_ratio.set_xlabel('t')
ax_ratio.set_ylabel('Ratio')
ax_ratio.set_ylim(0, 2)
ax_ratio.legend()
plt.show()

# Energy conservation

Add energy conservation of numerical method?

In [ ]:
n = 0  # choose trajectory index

# Position
x_NN = y_pred_generalization_NN[n].detach().cpu().numpy()
x_PINN = y_pred_generalization_PINN[n].detach().cpu().numpy()

# Time
t = test_generalization["t_tensor"][n].detach().cpu().numpy()

# Velocity via finite difference
v_NN = np.gradient(x_NN, t)
v_PINN = np.gradient(x_PINN, t)

#Energy conservation (check on generalized data)
E_t_NN = 0.5 * v_NN**2 + 0.5 * float(w) * x_NN**2
E0_NN = E_t_NN[0]
E_t_PINN = 0.5 * v_PINN**2 + 0.5 * float(w) * x_PINN**2
E0_PINN = E_t_PINN[0]

# diagnostics
abs_dev_NN = np.max(np.abs(E_t_NN - E0_NN))
rel_dev_NN = np.max(np.abs((E_t_NN - E0_NN) / E0_NN))
abs_dev_PINN = np.max(np.abs(E_t_PINN - E0_PINN))
rel_dev_PINN = np.max(np.abs((E_t_PINN - E0_PINN) / E0_PINN))

print(f"Initial energy NN E0 = {E0_NN:.6e}")
print(f"Max absolute NN deviation = {abs_dev_NN:.6e}")
print(f"Max relative NN deviation = {rel_dev_NN:.6e}")

print(f"Initial energy NN E0 = {E0_PINN:.6e}")
print(f"Max absolute NN deviation = {abs_dev_PINN:.6e}")
print(f"Max relative NN deviation = {rel_dev_PINN:.6e}")

plt.plot(test_generalization["t_tensor"][n], E_t_NN, label='E(t)', color="blue")
plt.plot(test_generalization["t_tensor"][n], E_t_PINN, label='E(t)', color="red")
plt.hlines(E0_NN, sol.t[0], sol.t[-1], colors='k', linestyles='dashed', label='E0')
#plt.hlines(E0_PINN, sol.t[0], sol.t[-1], colors='k', linestyles='dashed', label='E0') #both same EO?
plt.xlabel('t')
plt.ylabel('Energy')
plt.title('Energy vs time (check conservation)')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Computational time

In [ ]:
n=0

#Computation time
N = 200 #number of times we compute the time
timesteps_NN = np.zeros(N)
timesteps_PINN = np.zeros(N)
timesteps_Num = np.zeros(N)
N_list = np.arange(0,N)

X = test["X"][n]
y0 = [test["x0_tensor"][n][0], test["x0_tensor"][n][1]]  # initial condition [x0, v0]
t_span = test["t_tensor"][n][0], test["t_tensor"][n][-1]
w = test["args_tensor"][n]

#NN
for i, el in tqdm(enumerate(N_list), desc="Computation time NN: "):
    begin = time.perf_counter()
    for _ in range(el):
        model_NN(X)
    end = time.perf_counter()
    timesteps_NN[i] = end-begin

#PINN
for i, el in tqdm(enumerate(N_list), desc="Computation time PINN: "):
    begin = time.perf_counter()
    for _ in range(el):
        model_PINN(X)
    end = time.perf_counter()
    timesteps_PINN[i] = end-begin

#Numerical
for i, el in tqdm(enumerate(N_list), desc="Computation time Numerical: "):
    begin = time.perf_counter()
    for _ in range(el):
        sp.integrate.solve_ivp(diff, t_span, y0, t_eval=test["t_tensor"][n], method='RK45', args=(w))
    end = time.perf_counter()
    timesteps_Num[i] = end-begin

In [ ]:
#Plot results computational time
plt.plot(N_list, timesteps_NN, color='blue', label="Neural Network")
plt.plot(N_list, timesteps_PINN, color='red', label="Physics Informed Neural Network")
plt.plot(N_list, timesteps_Num, color='green', label="Numerical Method")
plt.xlabel("Number of iterations N")
plt.ylabel("time t (s)")
plt.grid(True)
plt.legend()
plt.show()

# Test performance in function of number of Training data

Do this individually for NN and PINN. If needed we can just export the data separately and plot it then instead of saving the model